In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

In [3]:
import fastai
fastai.defaults.device = torch.device('cpu')

In [4]:
from fastai.callbacks import *

In [5]:
import myutils as my

In [6]:
path = Path('data/mnist_png/')
classes = list(range(9))
head = create_head(1024,10,lin_ftrs=[128])

In [7]:
tfms = get_transforms(do_flip=False)
data2 = ImageDataBunch.single_from_classes(path, classes, tfms=tfms, size=26)

In [8]:
learner = create_cnn(data2, models.resnet18, custom_head=head).load('stage-1-50')

In [9]:
flatten_model(learner.model)[-1].out_features

10

In [10]:
class StoreHook(Callback):
    def __init__(self, module):
        super().__init__()
        self.custom_hook = hook_output(module)
        self.outputs = []
        
    def on_batch_end(self, train, **kwargs): 
        if (not train): self.outputs.append(self.custom_hook.stored)

In [11]:
nn_module = learner.model[-1][-3]
learner.callbacks = [ StoreHook(nn_module) ]

In [12]:
nn_module

BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [13]:
for i in (path/'test').iterdir():
    print(i)

data/mnist_png/test/78.png
data/mnist_png/test/92.png
data/mnist_png/test/58.png
data/mnist_png/test/100.png
data/mnist_png/test/12.png
data/mnist_png/test/9.png
data/mnist_png/test/20.png
data/mnist_png/test/73.png
data/mnist_png/test/16.png
data/mnist_png/test/62.png
data/mnist_png/test/7.png


In [14]:
learner.callbacks[0].outputs = []
my.run_folder(path/'test', learner)

In [23]:
my.binary_code(learner)

In [24]:
learner.callbacks[0].outputs

[tensor([[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
          1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
          1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
          0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
          0, 0, 1, 0, 1, 0, 0, 1]], dtype=torch.uint8),
 tensor([[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1,
          1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
          1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
          0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
          0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
          0, 0, 1, 1, 1, 0, 0, 1]], dtype=torch.uint8),
 tensor([[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0

In [18]:
def similarity(ten1, ten2):
    return (ten1 == ten2).sum().float()/len(ten1)

In [19]:
for i in range(1,len(learner.callbacks[0].outputs)):
    print(similarity(learner.callbacks[0].outputs[i-1][0], learner.callbacks[0].outputs[i][0]))

tensor(0.8281)
tensor(0.7422)
tensor(0.5391)
tensor(0.5469)
tensor(0.8203)
tensor(0.5938)
tensor(0.6484)
tensor(0.6719)
tensor(0.6953)
tensor(0.4766)
